In [155]:
from pypdf import PdfReader
from pathlib import Path
import pandas as pd
import numpy as np
import glob, os
import re

In [156]:
data_df = pd.DataFrame()

pdf_search = Path("test").glob("*.pdf")
pdf_files = pdf_files = [str(file) for file in pdf_search]

In [157]:
data_df = pd.DataFrame()

pdf_search = Path("test").glob("*.pdf")
pdf_files = pdf_files = [str(file) for file in pdf_search]

def parse(file: str):
    reader = PdfReader(file)

    # printing number of pages in pdf file
    print("Number of pages: ")
    print(len(reader.pages))

    # creating a page object
    # page = reader.pages[0]

    for i in range (len(reader.pages)):
        page = reader.pages[i]
        text = page.extract_text()
        text = page.extract_text().replace("\n", " ").strip() 
        # print(page.extract_text())

        if i == 0: #collecting metadata
            if "doi" in text:
                doi = re.findall(r"^\S+", text)
                data_df["DOI"] = doi
            
            # pattern = r"(?:[A-Za-z ]+\s(?:\(\d{4}\)|\d{4});?\s?\d+(?:,\s\d+)?|[A-Za-z]+\s\(\d{4}\)),?\s?\d{1,4}(?:–|[-–])\d{1,4}"
            pattern = r"[A-Za-z ]+\s\(?\d{4}\)?[;,]?\s?\d+[–\-]?\d+"
            journal = re.findall(pattern, text)
            if journal:
                data_df["Journal"] = journal
            else:
                data_df["Journal"] = ["No matches found"] * len(data_df)

def main():
    for file in pdf_files:
        parse(file)
    
if __name__ == "__main__":
    main()


Number of pages: 
10
Number of pages: 
7
Number of pages: 
12
Number of pages: 
15


In [164]:
def parse(file: str, records):
    reader = PdfReader(file)
    print(f"Number of pages in {file}: {len(reader.pages)}")

    for i in range(len(reader.pages)):
        page = reader.pages[i]
        text = page.extract_text().replace("\n", " ").strip()
        
        if i == 0:  # Collecting metadata from the first page
            doi = re.findall(r"\b10\.\d{4,9}/\S+\b", text)
            pattern = r"[A-Za-z ]+\s\(?\d{4}\)?[;,]?\s?\d+(?:[:,]\s?\d+)?(?:–|\-)\d+"
            journal = re.findall(pattern, text)

            records.append({
                "DOI": doi[0] if doi else "No DOI found",
                "Journal": journal[0] if journal else "No matches found"
            })
    return records

def main():
    pdf_search = Path("test").glob("*.pdf")
    pdf_files = [str(file) for file in pdf_search]

    if not pdf_files:
        print("No PDF files found in the directory!")
        return

    records = []
    for file in pdf_files:
        records = parse(file, records)
    
    # Convert collected records into a DataFrame
    data_df = pd.DataFrame(records)
    print(data_df)

if __name__ == "__main__":
    main()


Number of pages in test/example4.pdf: 10
Number of pages in test/example2.pdf: 7
Number of pages in test/example3.pdf: 12
Number of pages in test/example1.pdf: 15
                                      DOI                      Journal
0                       10.1002/ana.20653             No matches found
1                            No DOI found             No matches found
2  10.1016/j.neuropsychologia.2010.04.021                CA 95618-5412
3                    10.1093/brain/awh400   Brain (2005), 128, 773–787
